# Project: Wrange and Analyse WeRateDogs Twitter data (part 1/2)


## Table of Contents:

1. [Introduction](#intro)
2. [Wrangling](#wrangling)     
3. [Clean tasks done](#cleaning)
4. [Outputs  ](#outputs)


<a id='intro'></a>
# 1. Introduction


This project focused on wrangling data from the WeRateDogs Twitter account using Python, documented in a Jupyter Notebook (wrangle_act.ipynb). This Twitter account rates dogs with humorous commentary. The rating denominator is usually 10, however, the numerators are usually greater than 10. They’re Good Dogs Brent wrangles WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. WeRateDogs has over 4 million followers and has received international media coverage.

WeRateDogs downloaded their Twitter archive and sent it to Udacity via email exclusively for us to use in this project. This archive contains basic tweet data (tweet ID, timestamp, text, etc.) for all 5000+ of their tweets as they stood on August 1, 2017.

The goal of this project is to wrangle the WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The challenge lies in the fact that the Twitter archive is great, but it only contains very basic tweet information that comes in JSON format. I needed to gather, assess and clean the Twitter data for a worthy analysis and visualization.

This report focuses on providing an overview of the wrangling and cleaning of the data which is gathered from 3 data sources (see par 1.1.).


## 1.1. Origin of Data sources

We gather 3 pieces of data:

### a. Enhanced Twitter Archive

The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced.". We manually downloaded this twitter_archive_enhanced.csv file by clicking the following [link](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv). 

### b. Image Predictions File

The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) hosted on Udacity's servers and we downloaded it programmatically using python Requests library AND the following URL of the file: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)

### c. Twitter API

Back to the basic-ness of Twitter archives: retweet count and favorite count are two of the notable column omissions. Fortunately, this additional data can be gathered by anyone from Twitter's API. Well, "anyone" who has access to data for the 3000 most recent tweets, at least. But we, because we have the WeRateDogs Twitter archive and specifically the tweet IDs within it, can gather this data for all 5000+.
In this project, I'll be using Tweepy to query Twitter's API for data included in the WeRateDogs Twitter archive. This data will include retweet count and favorite count.



<br><br>
## 1.2. Key Points for Data Wrangling

Before we start, here are a few points to keep in mind when data wrangling for this project:

    1) We only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

    2) Fully assessing and cleaning the entire dataset requires exceptional effort so only a subset of its issues (eight (8) quality issues and two (2) tidiness issues at minimum) need to be assessed and cleaned.

    3) Cleaning includes merging individual pieces of data according to the rules of tidy data.

    4) The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.

<br><br>

## 1.3.  Relation to other input and output files of the project

* inputfiles: see url's in paragraph 1.1.
  - `twitter_archive_enhanced.csv` file with basic data of the WeRateDogs tweeets
  - `image_predictons.tsv` file with dog breed predictions of neural network
* `wrangle_act v1.0_DIRK.ipynb` : this is the extensive notebook with all details about the data gathering, wrangling and cleaning


* output file with the wrangled data of this data wrangling/cleaning exercise of the former bullit:
  - twitter_overal_table pandas dataframe saved in the `twitter_archive_master.csv` file
  - twitter_overal_table pandas dataframe is also saved in the `bestofrt.db database` file



* the `wrangle_report_DIRK_KADIJK.ipynb` file of this report which describes the wrangling/cleaning efforts and outputs  (thus *THIS REPORT*)
* the `act_report_DIRK_KADIJK.ipynb` file which communicates the insight and visualizations produced from the wrangled data

<br><br>
<a id='wrangling'></a>
# 2. Wrangling 

Wrangling resulted in the following identified Quality and Tidiness issues:

### 2.1. Quality issues
Identied Quality issues while wrangling the data from the 3 data sources mentioned in par 1.1.:

##### `twitter_archive table`

- tweet_ID, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id should be of object (string) datatype, in stead of numeric type (int64 or float64) since numerical operations are not supposed to be applicable for them. [solved]
- timestamp and retweeted_status_timestamp should be of 'date' datatype (in stead of 'object') [solved]
- huge number of missing values in reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id [solved]

- source column contains html tags at beginning ('<a  ') and end ('/a> ') [solved]

- issues expanded_url column: 
  - expanded_urls contains same url multiple times (2, 3, )
  - column 'expanded_urls contains url's in non twitter.com domein (e.g.: https://gofundme.com/ydvmve-surgery-for-jax)  [solved]
  - 59 NaN values (null items) in expanded_url column 
  
- table contains rows with reply message in the text column -> in case of value in in_reply_to_status_id column, then there are no values in the columns 'url', 'dogname', and 'dogtype'.  [solved]
- table contains rows with retweets which implies duplicated data -> in case of value in column 'retweeted_status_id', then there are no values in the columns 'url', 'dogname', and 'dogtype' ... and this while retweets are not required to capture scores and create risk of duplicating scores  (note: no duplicates of tweet_id identified in twitter_archive table. [solved]

- rating_numerator and rating_denominator:
  - A substantial number of a too high (>50) rating_numerator value is caused by wrong extraction:
    - 5 cases are caused because the text is a reply text (to be identified via a non-NaN value in 2nd column). [solved]
    - also cases where a value (e.g. 9.75) is wrongly captured from the decimal part (=number after the dot). See for instance the indexes 343, 695, 763, 1712. [solved]
    - cases with several dogs on the photo, to be identified with a rating_denominator value which is explainable by the multiple of the # of dogs times the default 10 dominator value. See also the next cel with the value count of the denominator  [solved]
    - some cases are caused by extracting a number from a non-score part of the tweet text (e.g. index 516)
    - Note: be aware that also 2 cases are caused by a subjective huge rating by the rater
  - But also some of the low (<10) values of rating_numerator is caused by wrong extraction:
    - cases where a float value is wrongly extracted (the decimal value after the dot is captered); e.g. Bella with index 45 got the score 5 based on the cut off of decimal part of the 13.5 score.  [solved]
    - also some are caused because the text is a reply text (in case of value in 2nd column). [solved]
    - and some captured a number from a non-score part of the tweet text (index 387).
    - some are caused because animal is not a dog (bug a goat: see 229 and 765). No dog stage is then assigned. [solved]
    - Note: be aware that also quite some low rating_numerator values are caused by a less positive explicit opinion of the rater
  - 23 cases with a rating_denominator values not equal to 10 which are not accurate since value must be 10:
     - 2 cases with a denominator value of resp. 2 and 7 (so less than 10) are wrongly extracted. [solved]
     - 1 case because of reply message [solved]
     - rest of cases due to several dogs on the photo, which explains a rating_denominator value equal to the multiple of the # of dogs times 10 [solved]

- dog names have values which are not dognames as: 'None', or 'a', or 'an.' or 'O', 'my' or 'by'  ; note: these names are wrongly extracted from 'text' field
  - also one dog name with only capital letters ('JD')  [solved]
  - some dognames start with a lowercase letter ==> str.title  [solved]
  - 745 'None' values in column 'name'  [solved]
- dog stage classification:   
  - a lot of 'None' values in the 4 dog stage class columns doggo, pupper, puppo and floofer.  ==> should be 'Null'  [solved]
  - there are even 1976 rows have no dog stage classification score whatsoever (84% of all rows)  [partly solved]
  - multiple dog stages occurs such as 'doggo puppo', 'doggo pupper', 'doggo floofer' ; e.g. 12 rows with both 'doggo' and 'floofer' filled in as dog stage [solved] <br><br> 
  

##### `image predictions table` (image_df dataframe)
- column names not self explanatory (p1, p1_conf, p1_dog); e.g. p2 predicts the dog class   [solved]
- values of p1, p2 and p3 (dog breed classification) have same words with uppercase or lowercase, while they are of same breed category ==> should be lowercase (str.lower) [solved]
- dog breeds are not consistently named in p1,p2,p3 columns  
- tweet_id is of str datatype [solved] <br><br>

##### `retweet and favorite count table` based query on API (df_tweet_json dataframe) 
- tweet_id is an int64 (should be of string object type).  [solved]
- retweet_count and favorite_count are of string datatype, and should be integer   [solved]<br><br>

### 2.2. Tidiness issues
##### `twitter_archive` table
- The variable for the dog's stage (dogoo, floofer, pupper, puppo) is spread in different columns ==> can be combined ('melted') in a single 'dog_stage' column  [solved]

##### `image predictions` table (image_df dataframe)
- This data set is part of the same observational unit as the data in the archive_table df   [solved]
 

##### `retweet and favorite count` table based query on API (df_tweet_json dataframe) 
- This data set is also part of the same observational unit as the data in the archive_table _df    [solved]



<a id='cleaning'></a>
# 3. Cleaning tasks done

The following cleaning of the identified Quality and Tidiness issues is done:

* Convert tweet_ID, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id to the object (string) datatype: via the .astype(str) method

- Convert tweet_ID to the object (string) datatype: via the .astype(str) method

- Convert 'timestamp' and 'retweet_status_timestamp' to the object (string) datatype: via the .to_datetime method



* convert tweet_count and favorite_count to datatype integer

* replace 'None' values in dog stage columns to an empty '' value via .replace method

* Create one column 'stage' for the various dog types: doggo, floofer, pupper, puppo, 'doggo, puppo', 'doggo, pupper', 'doggo, floofer' ascolumn name ' type ' with the categorical dtype
* Then drop the obsolete dog stage columns.

* Drop rows with a non-null value in the column 'reply_to_status_id
* Drop the columns in_reply_to_status_id and in_reply_to_user_id


* drop rows with a non-null value in the column 'retweet_status_id
* drop the 3 retweet columns retweet_status_user_id and retweeted_status_timestamp



* Merge the table twitter_archive_clean with tweet_json_clean table, based on left inner join

* first change datatype of tweet_id in image_df_clean dataframe to str (object)
* merge the twitter_json_interim table with image_df_clean table, based on left inner join

* replace columnname (p1, p1_conf, p1_dog, p2 etc) with self explanatory text values


* Replace predicted names to same format via method .str.capitalize()
* Also convert the column name to same format via method .str.capitalize()


* Replace the 'names' with silly values as 'a', 'an', 'the', 'None' and other lower case words with NaN in name column by converting to lowercase + .replace method

* Extract source url in 'source' column via .split method

* Extract rating_numerator as .float value from text field

* filter out the rows with rating_demoninator not equal to 10
* filter out the rows with rating_numerator > 20

<a id='outputs'></a>
# 4. Outputs

The output of wrangling/cleaning task is the `twitter_overal_table` pandas dataframe and is saved in the following 2 files:

* the twitter_overall_table is saved to the `twitter_archive_master.csv file`
* the `bestofrt.db` database file

<br><br>
See the `act_report_DIRK_KADIJK_v1.1.ipynb` file if you are interested in the insights and visualizations produced from these wrangled data.

